# CLPS1520 Tutorial 5
## Functions covered in this tutorial: 
### `squeeze, gather, gpuArray`
## Concepts covered in this tutorial:
### Basic overview of MatConvNet, and Convolutional Neural Network structure in Matlab

Hello everyone, and welcome back to Tutorial 5! My name is Michele and we’re going to dive into MatConvNet today and get some basic practice with Convolutional Neural Networks (CNNs). 

***
###### Intro to Requirements:

For this assignment, we’ll be using **MatConvNet, a Matlab toolbox that allows us to implement Convolutional Neural Networks (CNNs).** Instructions on how to download and compile MatConvNet locally have been provided in the Assignment 3 instructions. Some of you will have to download Xcode for MacOSX, which will allow you to compile MatConvNet. It can be found here: https://itunes.apple.com/us/app/xcode/id497799835?ls=1&mt=12. For Windows, you may have to download Visual Studio 13 Community, which can be found here: https://go.microsoft.com/fwlink/?LinkId=691978&clcid=0x409. MatConvNet has already been compiled as a module for CPU and GPU on CCV, so all we have to do is call upon which module we would like to use. 

**This tutorial will be run using the CPU compiled version of MatConvNet, to make running this tutorial locally on your laptop more feasible.** However, for the actual assignment, you are welcome to use the GPU compiled MatConvNet provided in CCV. We will also go over some basic GPU programming in Matlab, to show you the benchmarks between CPU and GPU processing.

***
###### Background: CPU vs GPU:

You’ve seen me throw around the acronyms CPU and GPU, but what do they mean? CPU means central processing unit and GPU means graphics processing unit (also known as a VPU or visual processing unit). Your CPU is the processor that your computer has running on it (currently, it’s name probably includes “i5” or “i7” along with an “inspiron” or something like that), and it allows you computer to compute and run programs. Traditionally, the speed of your CPU is maxed by the speed at which a disk inside of it can spin (usually denoted by a 2.5GHz or similar number). The faster it spins, the more calculations it can do per second, the faster your computer can do things. Your graphics card is your GPU, and it’s what’s telling the pixels in your screen what to do. GPUs can be thought of as many little CPUs, and in our case, they’re useful because we take advantage of the redundancy of calculations in our code. 

For our use, think of this metaphor: We want to tie bowties. Using a CPU, would be like tying one bowtie after another. This is “serial processing”, doing one thing after another is completed, and the only way to do this faster is to shorten the time that it takes to tie each individual bowtie. Using a GPU, would be like tying five bowties at a time, also called “parallel processing”. In this example, you can see that using a GPU would be faster at solving our problem, because there is an upper limit to how fast we can tie each bowtie, and each step is redundant, it doesn’t matter if we tie bowtie 1 at the same time as bowtie 3 because the tying of bowtie 3 doesn’t depend on how bowtie 1 was tied. A lot of processing follows this redundancy, which is why we can use GPU programming to speed up our code. Of course, CPUs and GPUs are more complex than a simple metaphor about bowties, but this is the extent of what you’ll need to know for this class. 

***
###### What we’re doing in this tutorial:

As you have learned from class, CNNs are Artificial Neural Networks (also known as Multilayer Perceptrons) with architectures inspired by animal visual processing. Today, we’re going to use a pre-trained CNN and load it into the Matlab to look at its structural representation. Hopefully, by the end of this tutorial, you’ll be more comfortable with using the MatConvNet toolbox and working with CNNs.

We’re going to be using the pretrained model **imagenet-vgg-verydeep-16**, which you can read more about from:  http://arxiv.org/abs/1409.1556/. It is a CNN trained for large-scale image recognition on the ImageNet data set.

First we're going to set up the CPU-compiled MatConvNet module.

If you're accessing CCV via SSH, you'll want to load the module yourself before running the setup. Do this by typing in:
> ``module load matconvnet/1.0-beta20``

This will load the CPU compiled version of MatConvNet. For the GPU version, type in:
> ``module load matconvnet/1.0-beta20-gpu``

By running the cell below, you'll set up the CPU-compiled MatConvNet module on CCV for this notebook.

*Note:* You'll need to run this cell in the notebook every time you'd like to work with the MatConvNet toolbox.

In [ ]:
vl_setupnn

Now we'll load the CNN we'll be working with. Load `vgg16.mat` as the variable `net`.

Now we're going to preprocess the CNN for use with MatConvNet.

In [ ]:
net = vl_simplenn_tidy(net);

Now that we have a CNN, we should run an image through it to see if the image is correctly classified!

First we’ll load the image and then resize it so that it’s `224x224`. All of the images `vgg16` was trained with were `224x224`, and so this is the size of the image it can be run on.

In the cell below, load in the image `jellyfish.jpg` using `imread`. Then resize it using `imresize`, and convert it to single format by calling `single` on it.

Then run the last line that has already been provided for you. The last line subtracts the average of the images `vgg16` was trained on from the input image.

In [ ]:



input = bsxfun(@minus, input, net.meta.normalization.averageImage);

Now we are going to run our image through the CNN using `vl_simplenn`. This function takes in the `net`, and our `input`. In the cell below, run `vl_simplenn` on `net` and `input`, and call the result `output`.

The final line, which has already been provided, separates out the final classification results of the CNN on the input image. In this line, `gather` moves a distributed or gpuArray to the local workspace. 

In [ ]:
output = vl_simplenn(net,input);
classificationLayer = gather(output(end).x);

The following code, which was given in the assignment, allows you to see the RGB components of the image that the CNN is responding to.

In [ ]:
figure()
subplot(1,3,1); imagesc(output(1).x(:,:,1));
subplot(1,3,2); imagesc(output(1).x(:,:,2));
subplot(1,3,3); imagesc(output(1).x(:,:,3));

Finally, we can see the classification of the CNN. We take the maximum response out of the classification layer, and then visualize the label that corresponds to that response.

In [ ]:
[bestScore, bestLabel] = max(classificationLayer);                          % The score and label (index)
labels = net.meta.classes.description;                                      % Grabbing the label
fprintf('%s (%d), score %.3f \n', labels{bestLabel}, bestLabel, bestScore); % Printing the results

Now that we saw the CNN in action, feel free to play around with other images. Which images does the CNN classify well, and which ones does it not? Think about how the results differ from image to image, as well as the difference confidence score. Where do you think these differences originate from?

***

Another way you could have gotten the classification of the image would be by the code below.

In [ ]:
scores = squeeze(gather(output(end).x)) ;
[bestScore, best] = max(scores) ;
imagesc(input) ;
title(sprintf('%s (%d), score %.3f',net.meta.classes.description{best}, best, bestScore)) ;

In this code, `squeeze` is used to remove all singleton dimensions from an array. This is easier to understand with the small example below. Visualize the array A in the cell below, then call `squeeze` on the array and see what changed.

In [ ]:
A = repmat(1,[1,1,5])



Now that you've played around with the CNN from the outside, let's look at what is on the inside. Run the cell below to take a look at the various layers that make up `vgg16`.

In [ ]:
vl_simplenn_display(net)

What do you think of all the layers? Do you remember what operations the layers perform?

If you look at the structure `net`, you'll see it has two properties `layers` and `meta`. The `layers` are all of the operation layers you just saw, while `meta` contains extra details on the CNN that MatConvNet ignores when running the CNN. Let's have a look at what `layers` contains in the cell below. Call upon the first layer in `net` to see what is inside. Remember, that when accessing a `struct`, you should index with `{}` instead of `()`.

As you can see, each layer has sub properties that define the operations it performs. You can remember from class, that the `weights` sub property are determined by the training of the CNN on it's training dataset. Can you try and visualize the weights of the first layer of `vgg16` by accessing the `net.layers.weights` sub property? Do this in the cell below. 

Finally, we're going to go over some quick introduction to GPU programming in Matlab. The good news, is that Matlab makes this very easy. For the extent of GPU programming we'll be doing in this class, all you'll need to know is `gpuArray`. By simply calling this on an array, Matlab will know how to distribute the calculations of whatever operations you'd like to run on this array. To see this in action, run the two cells below. 

In [ ]:
t_start_gpuArray = tic;

gpuA = gpuArray(ones(10000,10000)*3);
gpuA = gpuA.^4;

t_gpuArray = toc(t_start_gpuArray);
disp(['gpuArray took ' num2str(t_gpuArray) ' seconds'])

In [ ]:
t_start_regArray = tic;

regA = ones(10000,10000)*3;
regA = regA.^4;

t_regArray = toc(t_start_regArray);
disp(['regArray took ' num2str(t_regArray) ' seconds'])

In the cells above, `tic` and `toc` are built in Matlab functions that allow you to time how long a section of code takes to run. Here they are extremely useful in benchmarking GPU vs CPU computations on the array A. Feel free to play around with the cells to compare different operations. In the example above, I am multiplying every value to the fourth power, but other Matlab built in operations will work just as well. A full list of GPU supported Matlab built in functions can be found here: https://www.mathworks.com/help/distcomp/run-built-in-functions-on-a-gpu.html

However, keep in mind that there is a limit to how much memory is available on the GPU, so try not to make the arrays you are working with too big.

***

And this is the conclusion of Tutorial 5. Hopefully you've learned more about CNNs and MatConvNet, and are ready to tackle Assignment 3. Good luck!